In [1]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopy.distance
import math

In [2]:
traffic_df = pd.read_csv("traffic_crime.csv")
traffic_df = traffic_df.set_index("stop_id")

business_df = pd.read_csv("business_alc_fight_df.csv", index_col=0)

reasonable_df = pd.read_csv("reasonable_crime.csv")
reasonable_df = reasonable_df.set_index("stop_id")


reasonable_df.drop_duplicates(keep="first",inplace=True) 
traffic_df.drop_duplicates(keep="first",inplace=True) 

In [5]:
business_df["ACTIVITY DESC"].value_counts()

JANITORIAL SERVICES                              1122
ALL OTH PROF, SCIENTIFIC & TECHNICAL SERVICES    1060
ALL OTHER PERSONAL SERVICES                       781
BUILDING, DEVELOPING & GENERAL CONTRACTING        687
CONSULTING SERVICES                               686
BEAUTY SALONS                                     617
OTHER PERSONAL SERVICES                           612
FULL-SERVICE RESTAURANTS                          610
PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES     591
ALL OTHER SPECIAL TRADE CONTRACTORS               555
INDEPENDENT ARTISTS, WRITERS & PERFORMERS         475
OTHER SERVICES TO BUILDINGS & DWELLINGS           450
MANAGEMENT CONSULTING SERVICES                    438
LEGAL SERVICES                                    418
OFFICES OF DENTISTS                               415
LANDSCAPING SERVICES                              414
TAXI SERVICE                                      406
ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES     370
OTH PROFESSIONAL/SCIENTIFIC/

In [6]:
alc_fight_df = reasonable_df.loc[reasonable_df["crime_type"].isin(["*fight / battery*", "*alcohol*"])]


In [8]:
business_df.loc[business_df["ACTIVITY DESC"] == "BEAUTY SALONS"]

,ACTIVITY DESC,lon,lat,2447,2449,2451,2455,2459,2475,2483,...,42876,42877,42905,42943,42952,43034,43078,43138,43179,43180
31,BEAUTY SALONS,-117.215960,32.752632,3.690096,13.369894,4.621809,4.928556,6.350813,4.916095,4.583983,...,0.346547,4.638478,3.988476,5.920894,2.640903,5.868766,4.986388,6.278262,2.951120,4.605615
304,BEAUTY SALONS,-117.139786,32.760933,1.760372,12.483591,4.358896,4.574579,7.301669,4.610514,4.350254,...,2.474690,5.239924,4.698568,6.386597,0.634021,6.404599,4.364729,6.595772,2.695956,4.976470
423,BEAUTY SALONS,-117.085140,32.931030,19.051506,6.482348,15.527228,15.180222,26.111281,15.239568,15.587217,...,20.022089,24.063011,23.506824,25.201649,18.272983,25.227840,14.929039,25.378087,21.467769,23.788986
441,BEAUTY SALONS,-117.246940,32.798440,6.251513,8.332892,0.515333,0.180232,11.494483,0.268174,0.576071,...,5.080106,9.721100,9.077248,11.002984,4.694913,10.966018,0.647512,11.327918,7.533983,9.619863
452,BEAUTY SALONS,-117.162720,32.708046,5.998002,18.294024,9.661057,9.959588,1.482089,9.951645,9.625203,...,5.194457,0.637286,1.202383,0.833191,6.404299,0.722611,9.962782,1.283935,3.279412,1.045688
457,BEAUTY SALONS,-117.120674,32.831936,8.219665,4.753113,5.041553,4.703952,15.130827,4.801287,5.112851,...,9.185508,13.087428,12.533073,14.230174,7.324213,14.252810,4.241675,14.418353,10.511445,12.819898
728,BEAUTY SALONS,-117.127690,32.749310,1.740278,13.772433,5.590946,5.825781,6.143290,5.852613,5.576051,...,2.628868,4.030078,3.538114,5.125245,1.788071,5.157097,5.654870,5.306576,1.399982,3.719575
749,BEAUTY SALONS,-117.129770,32.955880,21.838183,9.091237,18.044602,17.705746,28.717594,17.754773,18.100146,...,22.562957,26.719327,26.148351,27.881725,20.989104,27.897995,17.512493,28.080678,24.173639,26.471406
760,BEAUTY SALONS,-117.129560,32.730430,3.406636,15.847694,7.474252,7.740250,4.162980,7.751363,7.449429,...,3.532596,2.026955,1.666050,3.049683,3.869662,3.090746,7.645510,3.224821,0.682264,1.656246
1029,BEAUTY SALONS,-117.124480,32.763367,1.465799,12.230624,4.333134,4.519702,7.666500,4.566823,4.332031,...,2.939990,5.568623,5.052803,6.677789,0.236352,6.707923,4.246799,6.857768,2.952009,5.270582
